## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal(shape=[28*28, 100]))
        self.b1 = tf.Variable(tf.zeros([100]))
        self.W2 = tf.Variable(tf.random.normal(shape=[100, 10]))
        self.b2 = tf.Variable(tf.zeros([10]))

    
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28*28])
        
        h1 = tf.matmul(x, self.W1) + self.b1
        h1_relu = tf.nn.relu(h1)
        
        logits = tf.matmul(h1_relu, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [5]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 147.14862 ; accuracy 0.058933333
epoch 1 : loss 134.78345 ; accuracy 0.056966666
epoch 2 : loss 125.845184 ; accuracy 0.058266666
epoch 3 : loss 118.83418 ; accuracy 0.061816666
epoch 4 : loss 112.99085 ; accuracy 0.06526667
epoch 5 : loss 107.892975 ; accuracy 0.06983333
epoch 6 : loss 103.3025 ; accuracy 0.07351667
epoch 7 : loss 99.09047 ; accuracy 0.07728333
epoch 8 : loss 95.187515 ; accuracy 0.08106667
epoch 9 : loss 91.55407 ; accuracy 0.08445
epoch 10 : loss 88.159966 ; accuracy 0.088
epoch 11 : loss 84.98092 ; accuracy 0.0917
epoch 12 : loss 81.99512 ; accuracy 0.094566666
epoch 13 : loss 79.18537 ; accuracy 0.09753333
epoch 14 : loss 76.54221 ; accuracy 0.10028333
epoch 15 : loss 74.05362 ; accuracy 0.1037
epoch 16 : loss 71.70773 ; accuracy 0.10706667
epoch 17 : loss 69.49758 ; accuracy 0.11073333
epoch 18 : loss 67.41649 ; accuracy 0.11485
epoch 19 : loss 65.45417 ; accuracy 0.117916666
epoch 20 : loss 63.596607 ; accuracy 0.12185
epoch 21 : loss 61.834927 ; 